### Example 8: conditionals -- progressive evaluation.
In this, we have an array `x` and based on the value of `x[0]` we will take a decision about
`x[1]`.

Say, we want to replicate the control flow 
```python
if x[0] > 0:
    return x[1]
else:
    return x[2]
```

Note that this will also let us handle control flows of the form 
```python
if f(x[:k]) > 0:
    do something with x[k:]
else:
    do something else
```

Here, the trick will be simple and motivated by the examples before. 
1. We will have two possible outputs, one if `x[0] > 0` and the other if not.
2. We will choose between the two based on the `mask` that we create (based on `x[0]`).

In [1]:
onnx_model = "example8.onnx"

In [2]:
import torch
import torch.nn as nn
import numpy as np

In [3]:
class SampleNet(nn.Module):
    def __init__(self):
        super(SampleNet, self).__init__()
        self.m = 1
        self.c = 0
    
    def forward(self, x):
        b = torch.max(x)
        output1 = x[1]
        output2 = x[2]
        mask = x.gt(0).type(torch.int64) 
        out = mask[0]*output1 + (1 - mask[0])*output2
        return self.m * out + self.c

In [4]:
model = SampleNet()
model.eval()

SampleNet()

In [5]:
# the warmup stage
x = torch.LongTensor([-1, 2, 3, 4])
out = model(x)

In [6]:
out

tensor(3)

In [7]:
torch.onnx.export(
model,
x,  # warming up the model
onnx_model,
opset_version=11,
do_constant_folding=True,
input_names=['input'],
output_names=['output'], 
verbose=True)

graph(%input : Long(4:1)):
  %1 : Long() = onnx::Constant[value={1}]()
  %2 : Long() = onnx::Gather[axis=0](%input, %1) # <ipython-input-3-de647dced85d>:9:0
  %3 : Long() = onnx::Constant[value={2}]()
  %4 : Long() = onnx::Gather[axis=0](%input, %3) # <ipython-input-3-de647dced85d>:10:0
  %5 : Long() = onnx::Constant[value={0}]()
  %6 : Bool(4:1) = onnx::Greater(%input, %5) # <ipython-input-3-de647dced85d>:11:0
  %7 : Long(4:1) = onnx::Cast[to=7](%6) # <ipython-input-3-de647dced85d>:11:0
  %8 : Long() = onnx::Constant[value={0}]()
  %9 : Long() = onnx::Gather[axis=0](%7, %8) # <ipython-input-3-de647dced85d>:12:0
  %10 : Long() = onnx::Mul(%9, %2) # <ipython-input-3-de647dced85d>:12:0
  %11 : Long() = onnx::Constant[value={0}]()
  %12 : Long() = onnx::Gather[axis=0](%7, %11) # <ipython-input-3-de647dced85d>:12:0
  %13 : Long() = onnx::Constant[value={1}]()
  %14 : Long() = onnx::Sub(%13, %12) # C:\Users\sambroy\AppData\Local\Continuum\anaconda3\envs\pt16\lib\site-packages\torch\tensor.p

In [8]:
# uncomment to install netron.
#!pip install netron
import netron
netron.start(onnx_model, port=8086)

Serving 'example8.onnx' at http://localhost:8086


### use the onnx model

In [9]:
import onnxruntime as ort
import numpy as np
sess = ort.InferenceSession(onnx_model)

In [10]:
sess.get_inputs()[0].name

'input'

In [11]:
# check out the signature of sess.run - it has to have the output, then something like a feed_dict.

# either pass the feed dict directly.
# passing an array of any other size will result in an error.
outs = sess.run(['output'],
                       {
                         'input': np.array([22, -22, 22, 12], dtype=np.int64)  
                       })

print(outs) 

[array(-22, dtype=int64)]


### Comments
While this works, this is not really what we wanted. Here, we evaluate `x[1]` and `x[2]` regardless of the 
value of `x[0]`. Can we avoid one of these steps?